In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
import os
import json
import warnings
warnings.filterwarnings('ignore')
os.environ["WANDB_DISABLED"] = "true"

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
json_file_path = "data"

with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"📊 Loaded {len(data)} questions")

In [ ]:
def format_conversation_new(item):
    question = item['cau_hoi']
    choices_text = ""
    for i, choice in enumerate(item['lua_chon']):
        choices_text += f"{choice}\n"
    choices_text = choices_text.strip()

    text = f"""<|im_start|>system
Bạn là một chuyên gia về trí tuệ nhân tạo. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, chọn đáp án đúng và giải thích chi tiết.
<|im_start|>user
### Câu hỏi: {question}
### Các lựa chọn:
{choices_text}
### Câu trả lời:
<|im_start|>assistant
Đáp án đúng là: {item['dap_an_dung']}
Giải thích: {item['giai_thich']}
""".strip()

    return text

# Format tất cả conversations
conversations = [format_conversation_new(item) for item in data]
print(f"✅ Formatted {len(conversations)} conversations")
print(conversations[0])


# Chia 85% train, 15% validation
train_conversations, val_conversations = train_test_split(
    conversations, test_size=0.15, random_state=42
)

print(f"📂 Train: {len(train_conversations)}, Validation: {len(val_conversations)}")

In [ ]:
model_name = "Qwen/Qwen3-1.7B"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    print("✅ Model loaded successfully")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("🔄 Try restarting kernel and running again")

In [ ]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_dataset = SimpleDataset(train_tokenized)
val_dataset = SimpleDataset(val_tokenized)
print(f"✅ Dataset created with {len(train_dataset)} samples")
print(f"✅ Dataset created with {len(val_dataset)} samples")

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)

# In số lượng tham số trước khi apply LoRA
def print_trainable_parameters(model):
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"➡️  Trainable parameters: {trainable} / {total} ({100 * trainable / total:.2f}%)")

print("📌 Trước khi apply LoRA:")
print_trainable_parameters(model)

# Apply LoRA vào model
model = get_peft_model(model, lora_config)

print("\n📌 Sau khi apply LoRA:")
print_trainable_parameters(model)

In [ ]:
training_args = TrainingArguments(
    output_dir="/kaggle/working/qwen_SICT",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    save_steps=500,
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_steps=10,
    fp16=True,
    remove_unused_columns=False,
    report_to=None
)

In [ ]:
def simple_collator(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=simple_collator,
    tokenizer=tokenizer,
)

print("✅ Trainer ready")

# CELL 10: Train
print("🚀 Starting training...")
try:
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained("/kaggle/working/qwen_SICT")
    print("🎉 Training completed!")
except Exception as e:
    print(f"❌ Training error: {e}")
    print("💡 Try reducing batch_size or max_length further")